In [4]:
import numpy as np
import pandas as pd
import json
import h5py
from PIL import Image
from keras import Input
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing import image 
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model,load_model
from keras.layers import Dense, Dropout, Concatenate
from keras.optimizers import SGD
import requests
from collections import Counter

In [17]:
from sklearn.model_selection import train_test_split
# create encode decode
output_voc_size = 1000
dataset = pd.read_csv('dataset.csv')
train = dataset.sample(frac=0.8)
test = dataset.drop(train.index)

In [25]:

images = train["urlImage"].values
tags = train["tags"].values
images_test = test["urlImage"].values
tags_test = test["tags"].values
demo = tags
demo = [x[1:-1].split("\'")  for x in demo ]
corpus = []
for x in demo:
    for y in x:
        if (y.find('#') != -1):
            corpus.append(y)
counts = Counter(corpus)
counts = sorted(counts, key=counts.__getitem__,reverse=True)

tag_to_num={}
numberOfTags=0
for a in counts[0:output_voc_size]:
    if not(a in tag_to_num):
        tag_to_num[a]=numberOfTags
        numberOfTags+=1

import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus2 = []
for i in range(0,len(tags)):
    review = re.sub('[^a-zA-Z#!0-9]', ' ', tags[i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review]
    review = ' '.join(review)
    corpus2.append(review.split(' '))
    
corpus3 = []
for i in range(0,len(tags_test)):
    review = re.sub('[^a-zA-Z#!0-9]', ' ', tags_test[i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review]
    review = ' '.join(review)
    corpus3.append(review.split(' '))
    
roenum=0
sumi=0
tagsBinaryTable=np.zeros(shape=(tags.shape[0],output_voc_size))
y_test = np.zeros(shape=(tags_test.shape[0],output_voc_size))
for row in corpus2:
    for cul in row:
        if cul in tag_to_num:
            tagsBinaryTable[roenum,tag_to_num[cul]]=1
            sumi+=1
    roenum=roenum+1

roenum=0
sumi=0
for row in corpus3:
    for cul in row:
        if cul in tag_to_num:
            y_test[roenum,tag_to_num[cul]]=1
            sumi+=1
    roenum=roenum+1

In [30]:
from six.moves.urllib.request import build_opener
from six.moves.urllib_error import URLError
import scipy.misc
from io import BytesIO
target_size = (320,320)


def load_image_url(url, mode='RGB'): 
    opener = build_opener()
    opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
    file = BytesIO(opener.open(url).read())
    return scipy.misc.imread(file, mode=mode)

# def load_image_url(url):
#     response = requests.get(url)
#     img = Image.open(BytesIO(response.content))
#     return img

def load_images_for_epoch(urls,tags, index,batch_size):
    x_train = np.zeros((batch_size, 320,320,3))
    y_train = np.zeros((batch_size, 1000))
    bad_urls = []
    i = 0
    while i < batch_size or index >= len(tags):
        try:
            im = load_image_url(urls[index])
            im = Image.fromarray(np.uint8(im))
            if im.size != target_size:
                im = im.resize(target_size)
            x = image.img_to_array(im)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            x_train[i] = x
            y_train[i] = tags[index + i]
            i+=1
        except URLError:
            bad_urls.append(i)
        index += 1
    return x_train, y_train, index

      
# im = load_image_url(images[0])
# print(im.shape)
# im = Image.fromarray(np.uint8(im))
# img = im.resize(target_size)
# print(img)
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
# x


# def load_data(path='mnist.npz'):
#     """Loads the MNIST dataset.
#     # Arguments
#         path: path where to cache the dataset locally
#             (relative to ~/.keras/datasets).
#     # Returns
#         Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
#     """
#     path = get_file(path, origin='https://s3.amazonaws.com/img-datasets/mnist.npz')
#     f = np.load(path)
#     x_train, y_train = f['x_train'], f['y_train']
#     x_test, y_test = f['x_test'], f['y_test']
#     f.close()
#     return (x_train, y_train), (x_test, y_test)

In [27]:
video = Input(shape=(320,320,3),name='video')
cnn = InceptionV3(weights='imagenet',
                  include_top=False,
                  pooling='avg')(video)
cnn.trainable = False
# x = Concatenate([cnn])
# x = Dense(1000, activation='relu', dropout=0.5)(x)
predictions = Dense(output_voc_size, name='predictions')(cnn)

model = Model(video, predictions)
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
             metrics=['accuracy'])

In [32]:
from keras.models import load_model
model = load_model('my_model_1.h5')

batch_size=50
    
json_string = model.to_json()
with open('model.json', 'w') as outfile:
    json.dump(json_string, outfile)

# batch and latent size taken from the paper
epochs = 50
size = len(images)
index = 0
index_2 = 0 
for epoch in range(epochs):
    while size > index + batch_size:
        k = index / batch_size
        print("starting epoch: %d" % k)
        x,y,index = load_images_for_epoch(images,tagsBinaryTable, index,8*5)
        x_test,y_testing,index_2 = load_images_for_epoch(images_test, y_test, index_2,2*5)
        model.fit(x,y)
        score = model.evaluate(x_test,y_testing, verbose=1)
        print(score)
        model.save('my_model_2.h5')




starting epoch: 0
Epoch 1/1
10/10 [==============================] - 10s
[0.11296619474887848, 0.83959996700286865]
starting epoch: 0
Epoch 1/1
10/10 [==============================] - 5s
[0.1105773076415062, 0.85109996795654297]
starting epoch: 1
Epoch 1/1
10/10 [==============================] - 5s
[0.076173342764377594, 0.84419995546340942]
starting epoch: 2
Epoch 1/1
10/10 [==============================] - 5s
[0.079618498682975769, 0.83550006151199341]
starting epoch: 3
Epoch 1/1
10/10 [==============================] - 5s
[0.051891546696424484, 0.90450000762939453]
starting epoch: 4
Epoch 1/1
10/10 [==============================] - 5s
[0.072553932666778564, 0.83469998836517334]
starting epoch: 5
Epoch 1/1
10/10 [==============================] - 5s
[0.088060364127159119, 0.82080000638961792]
starting epoch: 6
Epoch 1/1
10/10 [==============================] - 6s
[0.084063827991485596, 0.84499996900558472]
starting epoch: 6
Epoch 1/1
10/10 [==============================] - 6s
[0

KeyboardInterrupt: 